This notebook visualizes the ACC data and goes over steps to find roll-pitch-yaw angels

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from scipy.io import loadmat
import os
import matplotlib.pyplot as plt


# Visualize ACC and Gyro data
Check a section of running data on both ACC and Gyro

In [ ]:
# Load an example ACC data
watch_df = pd.read_csv('../data/exported_csv/Subject_1_watch.csv')
acc_sampling_freq = 50
gyro_sampling_freq = 50

# plot the 3 axis ACC
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(6, 8))

color_values = ['blue', 'green', 'red']
axis_names = ['X', 'Y', 'Z']
time_value = np.arange(watch_df.shape[0])/acc_sampling_freq
for i in range(1,4):
    # Plot data in each subplot
    axs[i-1].grid(alpha = 0.25)
    axs[i-1].plot(time_value, watch_df[f'sigAcc_{i}'].values, color=color_values[i-1])
    axs[i-1].set_title(f'ACC {axis_names[i-1]}')
    axs[i-1].set_xlabel('Time, seconds')
    axs[i-1].set_xlim([190,200])
    # axs[i-1].set_ylim([25000,35000])
    
plt.tight_layout(h_pad=2.0)
plt.show()

# plot the same for Gyro 
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(6, 8))

color_values = ['blue', 'green', 'red']
axis_names = ['X', 'Y', 'Z']
time_value = np.arange(watch_df.shape[0])/gyro_sampling_freq
for i in range(1,4):
    # Plot data in each subplot
    axs[i-1].grid(alpha = 0.25)
    axs[i-1].plot(time_value, watch_df[f'sigGyro_{i}'].values, color=color_values[i-1])
    axs[i-1].set_title(f'Gyro {axis_names[i-1]}')
    axs[i-1].set_xlabel('Time, seconds')
    axs[i-1].set_xlim([190,200])
    
plt.tight_layout(h_pad=2.0)
plt.show()

# Convert the units
Based on the data ReadMe file the is a "6-axis inertial measurement unit (IMU; LSM6DSMUSTR, STMicroelectronics)".  
The data sheet can be found here: https://www.mouser.ca/datasheet/2/389/en.DM00218116-1107506.pdf  
The information in the datasheet is sued to convert the values from raw digital readings to acceleration and angular velocity units

## Coversion factor
Depending on the sensitivity set on the device different conversion factors need to be used. The overall formula is:  
Acceleration (m/s²) = raw_data × sensitivity (m/s² per LSB)  
To convert these to m/s² per LSB:
- ±2g:  0.061 × 9.80665 × e-3 ≈ 0.000598 m/s2/LSB
- ±4g:  0.122 × 9.80665 × e-3 ≈ 0.001196 m/s2/LSB
- ±8g:  0.244 × 9.80665 × e-3 ≈ 0.002392 m/s2/LSB
- ±16g: 0.488 × 9.80665 × e-3 ≈ 0.004784 m/s2/LSB

Since there is no information in the data ReadMe file about the resolution, I tried all 4 and decided ±2g makes sense and assumed that as the resolution.  
The readme mentioned that they added 32768 to the digital data to remove any negative value, with a ±2g this is 2 * 9.79 (2*g). So the assumption makes sense.  
  
For Gyro the conversion table is:  
- ±125°/s:   0.0000763 rad/s per LSB
- ±250°/s:   0.0001527 rad/s per LSB
- ±500°/s:   0.0003054 rad/s per LSB
- ±1000°/s:  0.0006109 rad/s per LSB
- ±2000°/s:  0.0012217 rad/s per LSB

In [ ]:
# convert ACC to m/s2
acc_convertion_factor = 0.000598
axis_names = ['X', 'Y', 'Z']
for i in range(1,4):
    watch_df[f'sigAcc_{axis_names[i-1]}_mps2'] = (watch_df[f'sigAcc_{i}'] - 32768) * acc_convertion_factor
# convert Gyro to r/s
gyro_conversion_factor = 0.0000763
axis_names = ['X', 'Y', 'Z']
for i in range(1,4):
    watch_df[f'sigGyro_{axis_names[i-1]}_rps'] = (watch_df[f'sigGyro_{i}'] - 32768) * gyro_conversion_factor

# visualize the converted acc and gyro values
# plot the 3 axis ACC
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(6, 8))
color_values = ['blue', 'green', 'red']
axis_names = ['X', 'Y', 'Z']
time_value = np.arange(watch_df.shape[0])/acc_sampling_freq
for i in range(1,4):
    # Plot data in each subplot
    axs[i-1].grid(alpha = 0.25)
    axs[i-1].plot(time_value, watch_df[f'sigAcc_{axis_names[i-1]}_mps2'].values, color=color_values[i-1])
    axs[i-1].set_title(f'ACC {axis_names[i-1]}, m/s2')
    axs[i-1].set_xlabel('Time, seconds')
    axs[i-1].set_xlim([190,200])
    # axs[i-1].set_ylim([25000,35000])
    
plt.tight_layout(h_pad=2.0)
plt.show()

# plot the same for Gyro 
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(6, 8))

color_values = ['blue', 'green', 'red']
axis_names = ['X', 'Y', 'Z']
time_value = np.arange(watch_df.shape[0])/gyro_sampling_freq
for i in range(1,4):
    # Plot data in each subplot
    axs[i-1].grid(alpha = 0.25)
    axs[i-1].plot(time_value, watch_df[f'sigGyro_{axis_names[i-1]}_rps'].values, color=color_values[i-1])
    axs[i-1].set_title(f'Gyro {axis_names[i-1]}, r/s')
    axs[i-1].set_xlabel('Time, seconds')
    axs[i-1].set_xlim([190,200])
    
plt.tight_layout(h_pad=2.0)
plt.show()

# Get Pitch and Roll
To get the pitch (theta) and roll (phi) values from acc and gyro
To do this I'm following this video serie on youtube by @PhilsLab
- https://www.youtube.com/watch?v=RZd6XDx5VXo
- https://www.youtube.com/watch?v=BUW2OdAtzBw
- https://www.youtube.com/watch?v=hQUkiC5o0JI

This series waks through the nessecity of using both Gyro and ACC for a better angular velocity estimaiton and covers complementry filter and extended Kalman filter to smooth the estimated values

In [ ]:
# to avoid zero devision, we add the 2g value back in for both ACC and Gyro and use the zero corrected version for this
axis_names = ['X', 'Y', 'Z']
for i in range(1,4):
    watch_df[f'sigAcc_{axis_names[i-1]}_mps2_zero_corrected'] = watch_df[f'sigAcc_{i}'] * acc_convertion_factor
# convert Gyro to r/s
for i in range(1,4):
    watch_df[f'sigGyro_{axis_names[i-1]}_rps_zero_corrected'] = watch_df[f'sigGyro_{i}'] * gyro_conversion_factor

# complementry filter
# constants
comp_filt_alpha = 0.05
g_value = 9.81
rad_to_degree = 57.2957795131
# initialization
phiHat_rad = 0
thetaHat_rad = 0

estimated_theta = np.zeros(watch_df.shape[0])
estimated_phi = np.zeros(watch_df.shape[0])

for i in range(watch_df.shape[0]):

    ax_acc = watch_df['sigAcc_X_mps2_zero_corrected'].values[i]
    ay_acc = watch_df['sigAcc_Y_mps2_zero_corrected'].values[i]
    az_acc = watch_df['sigAcc_Z_mps2_zero_corrected'].values[i]

    phiHat_acc_rad = np.arctan(ay_acc / az_acc)
    thetaHat_acc_rad = np.arctan(ax_acc / g_value)


    p_gyro = watch_df['sigGyro_X_rps_zero_corrected'].values[i]
    q_gyro = watch_df['sigGyro_Y_rps_zero_corrected'].values[i]
    r_gyro = watch_df['sigGyro_Z_rps_zero_corrected'].values[i]

    phiDot_gyro = p_gyro + np.tan(thetaHat_acc_rad) * (np.sin(phiHat_acc_rad) * q_gyro + np.cos(phiHat_acc_rad) * r_gyro) # roll
    thetaDot_gyro = np.cos(phiHat_acc_rad) * q_gyro - np.sin(phiHat_acc_rad) * r_gyro # pitch

    phiHat_rad = comp_filt_alpha * phiHat_acc_rad + ((1 - comp_filt_alpha) * (phiHat_rad + phiDot_gyro * (1 / gyro_sampling_freq)))
    thetaHat_rad = comp_filt_alpha * thetaHat_acc_rad + ((1 - comp_filt_alpha) * (thetaHat_rad + thetaDot_gyro * (1 / gyro_sampling_freq)))

    estimated_phi[i] = phiHat_rad * rad_to_degree
    estimated_theta[i] = thetaHat_rad * rad_to_degree

# plot estimated Roll (phi) and Pitch (theta)
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(6, 8))

color_values = ['green', 'red']
axis_names = ['Roll, degree', 'Pitch, degree']
time_value = np.arange(watch_df.shape[0])/acc_sampling_freq
values_list = [estimated_phi, estimated_theta]
for i in range(2):
    # Plot data in each subplot
    axs[i].grid(alpha = 0.25)
    axs[i].plot(time_value, values_list[i], color=color_values[i])
    axs[i].set_title(axis_names[i])
    axs[i].set_xlabel('Time, seconds')
    axs[i].set_xlim([190,200])

plt.tight_layout(h_pad=2.0)
plt.show()